<a href="https://colab.research.google.com/github/Karn2898/GenAI_Learning_Archive/blob/main/text_summarizer_with_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [1]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 5.0.0
Uninstalling transformers-5.0.0:
  Successfully uninstalled transformers-5.0.0
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 24.1 MB/s eta 0:00:00


In [7]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset
import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
import torch

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_ckp="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model_ckp)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [12]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckp).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/680 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
PegasusForConditionalGeneration LOAD REPORT from: google/pegasus-cnn_dailymail
Key                                  | Status  | 
-------------------------------------+---------+-
model.decoder.embed_positions.weight | MISSING | 
model.encoder.embed_positions.weight | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [17]:
dataset = load_dataset("xsum")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [18]:
dataset["train"][0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [20]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})


In [23]:
split_lengths = [len(dataset[split])for split in dataset]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nDocument:")

print(dataset["test"][1]["document"])

print("\nSummary:")

print(dataset["test"][1]["summary"])

Split lengths: [204045, 11332, 11334]
Features: ['document', 'summary', 'id']

Document:
Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday.
Detectives said three firearms, ammunition and a five-figure sum of money were recovered.
A 26-year-old man who was arrested and charged appeared at Edinburgh Sheriff Court on Thursday.

Summary:
A man has appeared in court after firearms, ammunition and cash were seized by police in Edinburgh.


In [28]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['document'] , max_length = 1024, truncation = True )
    target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [29]:
dataset_pt = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [30]:
dataset_pt["train"]["input_ids"][1]

[99,
 1213,
 5826,
 584,
 196,
 31,
 6,
 6522,
 6350,
 12,
 3990,
 1308,
 31,
 57,
 79743,
 3039,
 54390,
 21,
 1224,
 8,
 1518,
 92,
 946,
 9,
 755,
 6,
 1118,
 4,
 295,
 54,
 5129,
 730,
 54,
 1045,
 6,
 125,
 7287,
 5,
 13659,
 374,
 18,
 1743,
 18,
 3472,
 12,
 6,
 336,
 1566,
 5,
 46112,
 38,
 17226,
 4,
 511,
 10,
 6,
 1370,
 1108,
 14,
 32,
 2476,
 5,
 6,
 73,
 32,
 1121,
 8,
 9594,
 4,
 65,
 37,
 50,
 108,
 462,
 12,
 3598,
 3257,
 4,
 36,
 1609,
 10,
 53,
 10,
 6,
 7287,
 140,
 120,
 10,
 6,
 4725,
 93,
 415,
 407,
 11971,
 21,
 939,
 8,
 116,
 93,
 71,
 6384,
 4,
 2492,
 1108,
 30,
 7116,
 2305,
 5543,
 8,
 35,
 1110,
 50,
 1370,
 10,
 6,
 2420,
 3579,
 575,
 94,
 54,
 93,
 2668,
 4,
 3282,
 30,
 19337,
 15,
 154,
 57,
 6,
 2178,
 4,
 119,
 12166,
 1660,
 15909,
 140,
 48,
 95,
 259,
 2269,
 27,
 474,
 6,
 1213,
 444,
 262,
 53,
 10,
 6,
 7287,
 166,
 8458,
 9,
 6,
 350,
 4,
 95,
 1538,
 6,
 2912,
 904,
 14,
 206,
 262,
 3141,
 5,
 23,
 14,
 563,
 17,
 6,
 1213,
 37,
 444,
 1

In [31]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [35]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    eval_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [37]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["train"],
                  eval_dataset=dataset_pt["validation"])

In [ ]:
trainer.train()